In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- Configuration ---
INPUT_FILE = "/content/drive/MyDrive/DiffPriv/Enron/selected_emails.json"   # Output from the previous script
OUTPUT_JSON_FILE = "/content/drive/MyDrive/DiffPriv/Enron/enron_email_analysis.json"
PROGRESS_FILE = "/content/drive/MyDrive/DiffPriv/Enron/enron_analysis_progress.txt"

In [ ]:
import google.generativeai as genai
import json
import os
import time
import sys

def configure_api_and_test():
    """
    Configures the genai API key from Colab Secrets or env variables
    and runs a simple test prompt to confirm it's working.
    """
    API_KEY = None
    try:
        # Check if running in Google Colab
        import google.colab.userdata
        API_KEY = google.colab.userdata.get('GEMINI_API_KEY')
        if not API_KEY:
            print("="*50)
            print("ERROR: 'GEMINI_API_KEY' not found in Colab Secrets.")
            print("Please click the 'Key' (🔑) icon in the left sidebar and add a new secret named 'GEMINI_API_KEY'.")
            print("="*50)
            sys.exit(1)
        print("Successfully loaded API key from Colab Secrets.")

    except ImportError:
        # Not in Colab, fall back to standard environment variables
        try:
            API_KEY = os.environ["GEMINI_API_KEY"]
            print("Successfully loaded API key from environment variable.")
        except KeyError:
            print("="*50)
            print("ERROR: 'GEMINI_API_KEY' not found in environment variables.")
            print("If not in Colab, please set the environment variable (e.g., 'export GEMINI_API_KEY=YOUR_KEY')")
            print("="*50)
            sys.exit(1)
    except Exception as e:
        print(f"Error loading API key: {e}")
        sys.exit(1)

    # Configure the API
    try:
        genai.configure(api_key=API_KEY)
    except Exception as e:
        print(f"Error configuring Generative AI: {e}")
        sys.exit(1)

# --- Prompts ---

SYSTEM_PROMPT = """
You are an AI assistant specializing in corporate data forensics, e-discovery, and financial linguistic analysis.
Your task is to analyze the text of a corporate email from the Enron corpus and extract specific Privacy and Utility features.
"""

USER_PROMPT_TEMPLATE = """
Analyze the following corporate email. Return a single, minified JSON object.
Use "N/A" if a feature is not found.

---
### **FEATURE DEFINITIONS**
---

#### 1. Privacy Features (PII & Sensitive Data)
* `Name`: Full names of employees or external contacts (e.g., "Jeff Skilling", "Andrew Fastow").
* `Email Address`: All email addresses found.
* `Phone Number`: Direct lines, extensions, or mobile numbers.
* `Employment Details`: Job titles, specific desk assignments (e.g., "West Power Trading Desk").
* `Location/Address`: Physical office locations or meeting addresses.
* `Social Security/ID`: (Rare) Any SSN, employee ID numbers, or passport numbers.
* `Timestamp`: The date/time of the email.

#### 2. Utility Features (Corporate Insight)
* `Category`: Classify into one of: [Finance, Trading, Legal, HR, IT, Personal, Logistics/Admin].
* `Topic`: A brief 3-5 word subject summary (e.g., "California Gas Capacity", "Ferris Wheel Deal Structure").
* `Summary`: A 1-2 sentence **PII-free** summary of the email content.
* `Financial Data`: Extract specific monetary figures, revenue mentions, or ticker symbols (e.g., "$50M", "ENE", "mark-to-market").
* `Project/Deal Name`: Internal code names for deals or partnerships (e.g., "Raptor", "LJM", "Project Jedi", "Dabhol").
* `Tone`: The professional tone (e.g., "Formal", "Aggressive", "Casual", "Urgent", "Hostile").
* `Action Item`: Specific requests or tasks (e.g., "Review the attached spreadsheet", "Call the counterparty").

---
### **EMAIL TO ANALYZE**
---
{email_context}
"""

def analyze_single_email(email_obj):
    email_body = email_obj.get('body_text')
    person_id = email_obj.get('person', 'Unknown')

    if not email_body:
        return None

    context_string = f"""
    Sender/Folder Info: {email_obj.get('folder', 'N/A')}
    Fintech Score: {email_obj.get('fintech_score', 'N/A')}

    Email Content:
    {email_body}
    """

    print(f"  > Analyzing email for: {person_id} (Score: {email_obj.get('fintech_score')})...")

    # --- FIX 1: JSON Mode Enforced ---
    model = genai.GenerativeModel(
        model_name='gemini-2.5-flash',
        system_instruction=SYSTEM_PROMPT,
        generation_config={"response_mime_type": "application/json"}
    )

    full_prompt = USER_PROMPT_TEMPLATE.format(email_context=context_string)

    # --- FIX 2: Retry Loop for Rate Limits (429/503) ---
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = model.generate_content(full_prompt)
            response_text = response.text.strip()

            # Use json.loads directly; JSON mode handles the formatting
            json_object = json.loads(response_text)

            # Merge metadata
            json_object['original_metadata'] = {
                'person': person_id,
                'file_path': email_obj.get('file_path'),
                'fintech_score': email_obj.get('fintech_score')
            }
            return json_object

        except Exception as e:
            error_msg = str(e)
            # If rate limited or service unavailable, wait and retry
            if "429" in error_msg or "503" in error_msg:
                wait_time = (2 ** attempt) + 2  # Exponential backoff: 3s, 4s, 6s...
                print(f"    ! API Busy (Attempt {attempt+1}/{max_retries}). Retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"    ! Non-recoverable error: {e}")
                return None

    print(f"    ! Failed after {max_retries} attempts.")
    return None

def main():
    # 1. Load Input
    if not os.path.exists(INPUT_FILE):
        print(f"Input file {INPUT_FILE} not found. Please run the selection script first.")
        return

    with open(INPUT_FILE, 'r') as f:
        all_emails = json.load(f)

    # 2. Load Progress/Existing Results
    start_index = 0
    all_results = []

    if os.path.exists(OUTPUT_JSON_FILE):
        try:
            with open(OUTPUT_JSON_FILE, 'r') as f:
                all_results = json.load(f)
                start_index = len(all_results)
                print(f"Resuming from index {start_index}")
        except json.JSONDecodeError:
            print("Output file corrupt, starting over.")

    # 3. Process
    try:
        for i, email_obj in enumerate(all_emails[start_index:], start=start_index):
            print(f"Processing {i+1}/{len(all_emails)}...")

            result = analyze_single_email(email_obj)

            if result:
                all_results.append(result)
                if i % 5 == 0: # Save periodically
                    with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
                        json.dump(all_results, f, indent=4)

            # Base sleep to be polite to the API
            time.sleep(1)

    except KeyboardInterrupt:
        print("\nStopping early...")
    finally:
        with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, indent=4)
        print(f"Saved {len(all_results)} results to {OUTPUT_JSON_FILE}")

if __name__ == "__main__":
    # --- FIX 3: Actually calling the configuration function ---
    configure_api_and_test()
    main()

Successfully loaded API key from Colab Secrets.
Resuming from index 4
Processing 5/2061...
  > Analyzing email for: allen-p (Score: 41)...
Processing 6/2061...
  > Analyzing email for: allen-p (Score: 41)...
Processing 7/2061...
  > Analyzing email for: allen-p (Score: 41)...
Processing 8/2061...
  > Analyzing email for: allen-p (Score: 41)...
Processing 9/2061...
  > Analyzing email for: allen-p (Score: 41)...
Processing 10/2061...
  > Analyzing email for: allen-p (Score: 39)...
Processing 11/2061...
  > Analyzing email for: allen-p (Score: 39)...
Processing 12/2061...
  > Analyzing email for: allen-p (Score: 38)...
Processing 13/2061...
  > Analyzing email for: allen-p (Score: 36)...
Processing 14/2061...
  > Analyzing email for: allen-p (Score: 36)...
Processing 15/2061...
  > Analyzing email for: arnold-j (Score: 86)...
Processing 16/2061...
  > Analyzing email for: arnold-j (Score: 58)...
Processing 17/2061...
  > Analyzing email for: arnold-j (Score: 56)...
Processing 18/2061...


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 6482.43ms


Processing 1418/2061...
  > Analyzing email for: ruscitti-k (Score: 36)...
Processing 1419/2061...
  > Analyzing email for: ruscitti-k (Score: 35)...
Processing 1420/2061...
  > Analyzing email for: ruscitti-k (Score: 34)...
Processing 1421/2061...
  > Analyzing email for: ruscitti-k (Score: 34)...
Processing 1422/2061...
  > Analyzing email for: ruscitti-k (Score: 34)...
Processing 1423/2061...
  > Analyzing email for: ruscitti-k (Score: 34)...
Processing 1424/2061...
  > Analyzing email for: ruscitti-k (Score: 34)...
Processing 1425/2061...
  > Analyzing email for: ruscitti-k (Score: 34)...
Processing 1426/2061...
  > Analyzing email for: ruscitti-k (Score: 33)...
Processing 1427/2061...
  > Analyzing email for: ruscitti-k (Score: 33)...
Processing 1428/2061...
  > Analyzing email for: ruscitti-k (Score: 33)...
Processing 1429/2061...
  > Analyzing email for: ruscitti-k (Score: 33)...
Processing 1430/2061...
  > Analyzing email for: sager-e (Score: 63)...
Processing 1431/2061...
  > 

In [ ]:
!ls /content/drive/MyDrive/DiffPriv/Enron

data_extract.py		     Enron_Email_Analysis.ipynb  out.jsonl
email_analysis_results.csv   enron_email_analysis.json	 selected_emails.json
enron_analysis_progress.txt  out.csv


In [12]:
import os
import json
import pandas as pd

# Define the path to your CSV file
csv_file_path = '/content/drive/MyDrive/DiffPriv/Enron/enron_email_analysis.csv'

# Read the CSV into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Optional: Display the first few rows to confirm it loaded correctly
print(df.head())
# Define the output path for your JSON file
json_file_path = '/content/drive/MyDrive/DiffPriv/Enron/email_analysis_results.json'

# Convert the DataFrame to JSON format and save it to the specified path
df.to_json(json_file_path, orient='records', indent=4)

print(f"Successfully converted CSV to JSON and saved to {json_file_path}")



  original_metadata.person                                               Name  \
0                  allen-p                                        Candy Haase   
1                  allen-p  ['Matt Johnson', 'Albert Viscio', 'Soli Forouz...   
2                  allen-p  ['Phillip K. Allen', 'Matt Smith', 'Rebecca Ca...   
3                  allen-p                                                NaN   
4                  allen-p  ['Phillip K. Allen', 'Jeff Dasovich', 'Tim Bel...   

                                       Email Address  \
0  annualconference@prosrm.com, k..allen@enron.co...   
1          ['news@prosrm.com', 'k..allen@enron.com']   
2  ['k..allen@enron.com', 'matt.smith@enron.com',...   
3  ei_editor@ftenergy.com, einsighthtml@listserv....   
4  ['k..allen@enron.com', 'jeff.dasovich@enron.co...   

                               Phone Number  \
0  713-335-5151, 713-335-5253, 713-335-8144   
1          ['713-335-5151', '713-335-8144']   
2                        ['(719) 52